In [ ]:
import random
import numpy as np
import pandas as pd
from numpy import linalg as la
import matplotlib.pyplot as plt

SGD Strategy Output

In [ ]:
def results(B):
    # Calculating SGD Costs
    P = np.zeros(T); X = np.zeros(T)
    A = np.zeros(T)
    P[0] = 50; X[0] = 0
    A[0] = 0
    for t in range(1, T):
        X[t] = rho*X[t-1] + N[t]
        P[t] = P[t-1] + theta*B[t] + gamma*X[t] + E[t]
        A[t] = A[t-1] + P[t]*B[t]

    execution_cost = np.sum(P*B)
    total_shares = np.sum(B)
    
    return([B, X, execution_cost, total_shares])

Optimal Strategy

In [ ]:
def Optimum():
    # Initialize Arrays
    P = np.zeros(T); X = np.zeros(T)
    S = np.zeros(T); B = np.zeros(T)
    A = np.zeros(T)
    a = np.zeros(T-1); b = np.zeros(T-1); c = np.zeros(T-1)
    d = np.zeros(T-1); e = np.zeros(T-1); f = np.zeros(T-1)

    # Initial Values
    P[0] = 50; X[0] = 0
    S[0] = 100000; B[0] = 0
    A[0] = 0
    a[0] = theta; b[0] = gamma; c[0] = 0
    d[0] = 0; e[0] = 1; f[0] = 0

    # Calculating Parameters
    for i in range(1, T-1):
        a[i] = (theta*(i+2))/(2*(i+1))
        b[i] = gamma + (theta*rho*b[i-1])/(2*a[i-1])
        c[i] = (rho**2)*c[i-1] - ((rho**2)*(b[i-1]**2))/(4*a[i-1])
        d[i] = d[i-1] + c[i-1]*(0.001)
        e[i] = (1/(i+1))
        f[i] = (rho*b[i-1])/(2*a[i-1])

    # Calculating Optimal Costs
    for i in range(1, T):
        B[i] = e[T-i-1]*S[i-1] + f[T-i-1]*X[i-1]
        S[i] = S[i-1] - B[i]
        X[i] = rho*X[i-1] + N[i]
        P[i] = P[i-1] + theta*B[i] + gamma*X[i] + E[i]
        A[i] = A[i-1] + P[i]*B[i]
    return(B)

In [ ]:
# Initializing B with Constraint sum(B) = 100,000
T = 21
rv = np.random.normal(5000, 1000, T-1)
initial_B = np.insert(rv/np.sum(rv)*100000, 0, 0)

# Initializng White Noise
E = 0.125*np.random.normal(0, 0.125, T)
N =  0.125*np.random.normal(0, np.sqrt(0.001), T)
# Initializing Model Parameters
theta = 0.00005; gamma = 5; rho = 0.5

# Optimum
optimal_B = Optimum()
Optimum_results = results(optimal_B)

In [ ]:
print("Optimal Strategy")
print("Strategy Execution Costs", Optimum_results[2])
print("Standard Deviation per Period", np.std(Optimum_results[0])/20)

AdaGrad SGD

In [ ]:
def AdaGrad_SGD(B, learning_rate=0.025, num_iterations=10000, tol=1e-5):
    for i in range(1, T):
          eta = learning_rate
          G = np.zeros_like(B)
          res = np.zeros(num_iterations)
          for j in range(0, num_iterations):
            P = np.zeros(T); X = np.zeros(T)
            P[0] = 50; X[0] = 0
            for t in range(1, T):
                if t <= i:
                    X[t] = rho*X[t-1] + N[t]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t] + E[t]
                elif t > i:
                    X[t] = rho*X[t-1]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t]
            B[i:] -= eta*P[i:]/(np.sqrt(G[i:]) + 0.125)
            B[i:][B[i:] < 0] = 0
            B[i:] = B[i:]/sum(B[i:])*(100000-np.sum(B[:i]))
            res[j] = la.norm(P)
            print("Iter: {0:4d}, ".format(j), "Error: {0:6.3e}".format(res[j]))
            if res[j] <= tol:
                break
            else:
                G[i:] += P[i:]**2
    return B

In [ ]:
# Initializing B with Constraint sum(B) = 100,000
T = 21
initial_B = np.insert(rv/np.sum(rv)*100000, 0, 0)
# ADA_grad
optimal_B_AdaGrad = AdaGrad_SGD(initial_B)
AdaGrad_results = results(optimal_B_AdaGrad)

In [ ]:
print("AdaGrad Strategy")
print("Strategy Execution Costs", AdaGrad_results[2])
print("Excess Cost per Share", (AdaGrad_results[2] - Optimum_results[2])/100000 )
print("Increase in Standard Deviation per Share", np.std(AdaGrad_results[0]-Optimum_results[0])/20)

RMSprop SGD

In [ ]:
def RMSprop_SGD(B, learning_rate=0.025, beta1=0.98, num_iterations=10000, tol=1e-5):
    for i in range(1, T):
        eta = learning_rate
        G = np.zeros_like(B)
        res = np.zeros(num_iterations)
        for j in range(0, num_iterations):
            P = np.zeros(T); X = np.zeros(T)
            P[0] = 50; X[0] = 0
            for t in range(1, T):
                if t <= i:
                    X[t] = rho*X[t-1] + N[t]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t] + E[t]
                elif t > i:
                    X[t] = rho*X[t-1]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t]
            G[i:] = beta1*G[i:] + (1-beta1)*P[i:]**2
            B[i:] -= eta*P[i:]/(np.sqrt(G[i:]) + 0.125)
            B[i:][B[i:] < 0] = 0
            B[i:] = B[i:]/sum(B[i:])*(100000-np.sum(B[:i]))
            res[j] = la.norm(P)
            print("Iter: {0:4d}, ".format(j), "Error: {0:6.3e}".format(res[j]))
            if res[j] <= tol:
                break
    return B

In [ ]:
# Initializing B with Constraint sum(B) = 100,000
T = 21
initial_B = np.insert(rv/np.sum(rv)*100000, 0, 0)
# ADA_grad
optimal_B_RMSprop = RMSprop_SGD(initial_B)
RMSprop_results = results(optimal_B_RMSprop)

In [ ]:
print("RMSprop Strategy")
print("Strategy Execution Costs", RMSprop_results[2])
print("Excess Cost per Share", (RMSprop_results[2] - Optimum_results[2])/100000)
print("Increase in Standard Deviation per Share", np.std(RMSprop_results[0]-Optimum_results[0])/20)

Adam SGD

In [ ]:
def Adam_SGD(B, learning_rate=0.025, beta1=0.98, beta2=0.99, num_iterations=10000, tol=1e-5):
    for i in range(1, T):
        eta = learning_rate
        m = np.zeros_like(B)
        v = np.zeros_like(B)
        res = np.zeros(num_iterations)
        for j in range(0, num_iterations):
            P = np.zeros(T); X = np.zeros(T)
            P[0] = 50; X[0] = 0
            for t in range(1, T):
                if t <= i:
                    X[t] = rho*X[t-1] + N[t]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t] + E[t]
                elif t > i:
                    X[t] = rho*X[t-1]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t]
            m[i:] = beta1*m[i:] + (1-beta1)*P[i:]
            v[i:] = beta2*v[i:] + (1-beta2)*P[i:]**2
            m_hat = m[i:]/(1 - beta1**(j+1))
            v_hat = v[i:]/(1 - beta2**(j+1))
            B[i:] -= eta*m_hat/(np.sqrt(v_hat) + 1e-8)
            B[i:][B[i:] < 0] = 0
            B[i:] = B[i:]/sum(B[i:])*(100000-np.sum(B[:i]))
            res[j] = la.norm(P)
            print("Iter: {0:4d}, ".format(j), "Error: {0:6.3e}".format(res[j]))
            if res[j] <= tol:
                break
    return B

In [ ]:
# Initializing B with Constraint sum(B) = 100,000
T = 21
initial_B = np.insert(rv/np.sum(rv)*100000, 0, 0)
# Adam_SGD
optimal_B_Adam = Adam_SGD(initial_B)
Adam_results = results(optimal_B_Adam)

In [ ]:
print("Adam Strategy")
print("Strategy Execution Costs", Adam_results[2])
print("Excess Cost per Share", (Adam_results[2] - Optimum_results[2])/100000)
print("Increase in Standard Deviation per Share", np.std(Adam_results[0]-Optimum_results[0])/20)

Custom SGD

In [ ]:
def Custom_SGD(B, learning_rate=0.025, num_iterations=10000, tol=1e-5):
    for i in range(1, T):
        eta = learning_rate
        iterations = num_iterations
        res = np.zeros(iterations)
        for j in range(0, iterations):
            P = np.zeros(T); X = np.zeros(T)
            P[0] = 50; X[0] = 0
            for t in range(1, T):
                if t <= i:
                    X[t] = rho*X[t-1] + N[t]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t] + E[t]
                elif t > i:
                    X[t] = rho*X[t-1]
                    P[t] = P[t-1] + theta*B[t] + gamma*X[t]
            B[i:] -= eta*P[i:]
            B[i:][B[i:] < 0] = (100000-np.sum(B[:i]))/(T-i+1)
            B[i:] = B[i:]/sum(B[i:])*(100000-np.sum(B[:i]))
            res[j] = la.norm(P[i:])
            print("Iter: {0:4d}, ".format(j), "Error: {0:6.3e}".format(res[j]))
            if res[j] <= tol:
                break
            else:
                if res[j] >= np.average(res[max(0,j-100):j+1], weights=np.arange(1, len(res[max(0,j-100):j+1])+1)):
                    eta *= 0.5/num_iterations
                    iterations *= 2/num_iterations
                else:
                    eta *= 2/num_iterations
                    iterations *= 0.5/num_iterations
    return B

In [ ]:
# Initializing B with Constraint sum(B) = 100,000
T = 21
initial_B = np.insert(rv/np.sum(rv)*100000, 0, 0)
# Custom SGD
optimal_B_Custom = Custom_SGD(initial_B)
Custom_results = results(optimal_B_Custom)

In [ ]:
print("Custom Strategy")
print("Strategy Execution Costs", Custom_results[2])
print("Excess Cost per Share", (Custom_results[2] - Optimum_results[2])/100000 )
print("Increase in Standard Deviation per Period", np.std(Custom_results[0]-Optimum_results[0])/20)

Strategy Analysis

In [ ]:
data = {'Execution Costs': [Optimum_results[2], Custom_results[2], AdaGrad_results[2], RMSprop_results[2], Adam_results[2]],
        'Standard Deviation': [np.std(Optimum_results[0]), np.std(Custom_results[0]), np.std(AdaGrad_results[0]), np.std(RMSprop_results[0]), np.std(Adam_results[0])]}
df = pd.DataFrame(data, index=['Optimum', 'Custom', 'AdaGrad', 'RMSprop', 'Adam'])
df_sorted = df.sort_values(by='Execution Costs', ascending=True)
df_sorted['Rank'] = range(1, len(df_sorted) + 1)
print(df_sorted)

In [ ]:
plt.plot(optimal_B_Custom, label='Custom_SGD Strategy', color='red', linestyle=':')
plt.plot(optimal_B_AdaGrad, label='AdaGrad_SGD Strategy', color='orange', linestyle='--')
plt.plot(optimal_B_RMSprop, label='RMSprop_SGD Strategy', color='purple', linestyle=':')
plt.plot(optimal_B_Adam, label='Adam_SGD Strategy', color='blue', linestyle='--')
plt.plot(optimal_B, label='Optimal Strategy', color='black')
plt.legend()
plt.xlim(1,20); plt.xticks(range(1, 21))
plt.xlabel('Period')
plt.ylabel('Shares Bought')